# 引言

最简单的神经元：房价预测

<img src="images/最简单的神经元.png" width="300"/>

为什么用神经网络

<img src="images/为什么用神经网络.png" width="300"/>    

数据集越大，对越大的神经网络的性能越好。   
对于比较小的数据集，各种模型的性能差不多

# 第一部分 ： 二分类神经网络

### （一）样品的数据结构：

一张已经被标记为“xxx”的图 被称作样品

<img src="images/三色图.png" alt="image1" width="300"/>


图像由形状$（长度，高度，深度=3）$的3D阵列表示。然而，当您读取图像作为算法的输入时，您可以将其转换为形状为$（长度*高度*3，1）$的向量。    
换句话说，可以将三维阵列“展开”或重塑为一维矢量。
也就是把3个图层的数据连起来排成一列，成为一个  $x$    
具体方法如下：

In [ ]:
def image2vector(image):

    a=image.shape[0]
    b=image.shape[1]
    c=image.shape[2]
    v=image.reshape(a*b*c,1)

    return v

处理好图像后，得到向量x作为输入变量     

**输入**  
$x$: 表示一个 $n_x$ 维度特征，为输入特征，维度为 $(n_x, 1)$。


---

**输出**  
$y$: 表示输出结果，取值为 $(0,1)$;  
$(x^{(i)}, y^{(i)})$: 表示第 $i$ 组数据，可能是训练数据，也可能是测试数据，此处暂译为训练数据;

---

$X = [x^{(1)}, x^{(2)}, \dots, x^{(m)}]$: 表示不同的训练数据组成的输入矩阵，放在一个 $n_x \times m$ 的矩阵中;  
$Y = [y^{(1)}, y^{(2)}, \dots, y^{(m)}]$: 对应训练不同训练数据组成的输出矩阵，维度为 $1 \times m$。

表示测试集的时候，我们会用 $M$ 来默认表示 $𝑀_{train}$  ,而测试集  $𝑀_{test}$ 需要单独注明：   

$M = [(x^{(1)}, y^{(1)}),(x^{(2)}, y^{(2)})], \dots, (x^{(m)}, y^{(m)})]  $


### （2）逻辑回归（Logistic Regression）

对于二元分类问题来说，给定一个输入特征向量 $x$，它可能属于一类或另一类，模型的任务就是找出其属于哪一类。   
我们的模型在计算过程中，需要将输入特征 $x$ 转换为输出估计值 $\hat{y}$。    
比如对于猫图而言，如果“是猫图”的 $y$ 表示为 1， “不是猫图”的 $y$ 表示为 0   
那么  $\hat{y}$  需要在（0,1）之内，表示“是猫图”的可能性



在开始之前，我们先介绍一下用sigmoid函数来处理向量x

$$ \text{For } x \in \mathbb{R}^n \text{,     } sigmoid(x) = sigmoid\begin{pmatrix}
    x_1  \\
    x_2  \\
    ...  \\
    x_n  \\
\end{pmatrix} = \begin{pmatrix}
    \frac{1}{1+e^{-x_1}}  \\
    \frac{1}{1+e^{-x_2}}  \\
    ...  \\
    \frac{1}{1+e^{-x_n}}  \\
\end{pmatrix}\tag{1} $$

sigmoid函数的导数为：
$$sigmoid\_derivative(x) = \sigma'(x) = \sigma(x) (1 - \sigma(x))\tag{2}$$

In [ ]:
import numpy as np 

def sigmoid(x):
    s=1/(1+np.exp(-x))
    return s
def sigmoid_derivative(x):
    ds=sigmoid(x)*(1-sigmoid(x))    
    return ds

接下来就是预测的模型

$$
\hat{y}^{(i)} = \sigma(w^T x^{(i)} + b), \text{ where } \sigma(z) = \frac{1}{1+e^{-z}}
$$

Given $\{(x^{(1)}, y^{(1)}), \dots, (x^{(m)}, y^{(m)})\}$, 而我们希望 $\hat{y}^{(i)} \approx y^{(i)}$.


如何来衡量模型的准确性呢？  
需要用到 “损失函数”（loss function）：

$$ 
L(\hat{y},y) = -ylog(\hat{y})-(1-y)log(1-\hat{y})
$$

当 y=1 时，只有 $\hat{y}$ 尽可能大（收范围限制趋近于1），损失函数L才会小   
当 y=0 时，只有 $\hat{y}$ 尽可能小（收范围限制趋近于0），损失函数L才会小

然而这只是对于一个样本的衡量方法，对于一个样本集而言，需要进行累加，这称为 成本函数（Cost Function）

$$
J(w, b) = \frac{1}{m} \sum_{i=1}^m L(\hat{y}^{(i)}, y^{(i)})
$$
    

即
$$
J(w, b)= \frac{1}{m} \sum_{i=1}^m \left(-y^{(i)} \log(\hat{y}^{(i)}) - (1 - y^{(i)}) \log(1 - \hat{y}^{(i)})\right)
$$


那么现在优化的方向就已经相当明确了，我们要不断修改w和b来让成本函数尽量小   
而事实上，使用了sigmoid 函数，这个成本函数是有最低点的 



<img src='images/成本函数3D.png'><img>

所以直接使用 **梯度下降法**：

假设b不变时：
$$
w = w - a \frac{dJ(w)}{dw}
$$



其中      
𝑎 ：学习率（ learning rate）    
$a \frac{dJ(w)}{dw}$ : 步长 (step），即向下走一步的长度

注意：   
对于 $\frac{dJ(w)}{dw}$  我们一般简写做 $dw$

---

拓展到两个参数就是：
$$
w := w - a \frac{\partial J(w, b)}{\partial w}, \quad b := b - a \frac{\partial J(w, b)}{\partial b}
$$


    

### （3）逻辑回归中的梯度下降（Logistic Regression Gradient Descent）

我们在前面已经了解了逻辑回归的训练过程    

那么怎么计算 $\frac{\partial J(w, b)}{\partial w}$  和  $\frac{\partial J(w, b)}{\partial b}$   呢？

假设样本有2个特征 $x_1,x_2$，那么 z 的表达式应该修改为：

$$
z=w_1x_1+w_2x_2+b
$$


回忆一下：   
$\hat{y}^{(i)} = \sigma(w^T x^{(i)} + b) = a, \text{ where } \sigma(z) = \frac{1}{1+e^{-z}}$  

其损失函数为：   
$L(\hat{y},y) = -ylog(\hat{y})-(1-y)log(1-\hat{y})$    

对于单个样本而言，代价函数 $J(w,b) $ 就是损失函数：   
$L(a,y) = -ylog(a)-(1-y)log(1-a)$    

其中𝑎是逻辑回归的输出， 𝑦是样本的标签值

求导可得：   
$$ \frac{dL(a,y)}{da} = -\frac{y}{a}+\frac{1-y}{1-a} $$




同时：
$$
\begin{align*}
a &= \sigma(z) = \frac{1}{1+e^{-z}} \\
\implies 1 + e^{-z} &= \frac{1}{a} \\
\implies -e^{-z} \, dz &= -\frac{1}{a^2} \, da \\
\implies \frac{da}{dz} &= a^2 e^{-z} \\
&= a^2 \left( \frac{1}{a} - 1 \right) \\
&= a(1-a)
\end{align*}
$$


因为：   
$$ 
\frac{dL}{dz}  =\frac{dL}{da} \frac{da}{dz}     \\
$$
$$
\implies \frac{dL}{dz} =(-\frac{y}{a}+\frac{1-y}{1-a}) *  a(1-a)
$$
$$
\implies \frac{dL}{dz}= a-y
$$

同时，我们一般直接用 $dz$ 来表示 $\frac{dL}{dz}$

因为：
$$ 
\frac{dL}{dw}  =\frac{dL}{dz} \frac{dz}{dw}     \\
$$
$$
\implies \frac{dL}{dw} =(a-y)(x)
$$

所以：   
$\frac{dL}{dw_1} =(a-y)(x_1)$    

$\frac{dL}{dw_2} =(a-y)(x_2)$


因为：
$$ 
\frac{dL}{db}  =\frac{dL}{dz} \frac{dz}{db}     \\
$$
$$
\implies \frac{dL}{db} =a-y
$$

综上，对于单个样本的梯度下降算法：    
$$
w_1=w_1-adw_1  \\
w_2=w_2-adw_2   \\
b=b-adb
$$
其中
$$
dw_1 =(a-y)(x_1) \\
dw_2 =(a-y)(x_2)\\
db=a-y\\

$$

上述结论拓展到m个样本应该如何呢？     
我们知道：
$J(w, b) = \frac{1}{m} \sum_{i=1}^m L(\hat{y}^{(i)}, y^{(i)})  $     
即：  
$J(w, b) = \frac{1}{m} \sum_{i=1}^m L(a^{(i)}, y^{(i)})$    
其中 $a^{(i)}$ 是第 i 个样本的预测输出，$y^{(i)}$是第 i 个样本的实际标签。








对权重 \(w_1\) 的梯度：   
$$
\frac{\partial J}{\partial w_1} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)}) x_1^{(i)}
$$

对权重 \(w_2\) 的梯度：
$$
\frac{\partial J}{\partial w_2} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)}) x_2^{(i)}
$$

对偏置 \(b\) 的梯度：
$$
\frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)})
$$

梯度下降更新规则
使用上述计算出的平均梯度来更新参数：
$$
w_1 = w_1 - \alpha \frac{\partial J}{\partial w_1} \\
w_2 = w_2 - \alpha \frac{\partial J}{\partial w_2} \\
b = b - \alpha \frac{\partial J}{\partial b}
$$

其中，$\alpha$ 是学习率，用于控制更新步骤的大小。

### （四）向量化（Vectorization）

在上面的计算方法中,每进行一次梯度下降时，都需要使用for循环来遍历每一个样本    
事实上这样的效率非常低，所以我们学习使用 **向量化** 来解决这个问题

所谓向量化，就是对于一串数据，比如数组或者矩阵，本来采用for循环进行一个个的运算，    
但是现在使用numpy库的自带功能，去掉显式的for循环     


In [12]:
import numpy as np
import time

data = np.random.rand(1000000)  # 假设有一个大型数组

# 向量化计算平方根和平均值
tik = time.time()
sqrt_data_vectorized = np.sqrt(data)
tok = time.time()
time1 = tok - tik
print('time:', time1)
print(sqrt_data_vectorized[:5])  # 打印前5个元素作为示例


# 普通计算方法
tik = time.time()
sqrt_list = []  # 使用列表收集平方根结果
for i in data:
    sqrt_list.append(i**0.5)  # 向列表追加平方根
sqrt_data = np.array(sqrt_list)  # 将列表转换为NumPy数组
tok = time.time()
time2 = tok - tik    
print('time:', time2)
print(sqrt_data[:5])  # 打印前5个元素作为示例


Vectorized computation time: 0.0019941329956054688
[0.88992777 0.75336519 0.87875718 0.93422771 0.88558512]
Loop computation time: 0.12228894233703613
[0.88992777 0.75336519 0.87875718 0.93422771 0.88558512]


上面的结果可以看到相差了100多倍    
注意：  
所谓向量化，不一定要调用numpy的计算函数比如 np.mean() np.sqrt()等，而是使用了np.array 的数据格式   
比如对于一个 np.array 进行 **2 平方操作，也是向量化的

根据我们已经学会的算法，可以知道现在的计算过程如下：   

初始化 $J=0$, $dw_1=0$, $dw_2=0$, $db=0$。代码流程如下：

```python
J = 0; dw1 = 0; dw2 = 0; db = 0
for i in range(1, m+1):  # 假设m是样本数量
    z_i = w * x[i] + b  # 这里假设x[i]是一个包含x1和x2的向量
    a_i = sigmoid(z_i)
    J += -[y[i] * log(a_i) + (1 - y[i]) * log(1 - a_i)]
    dz_i = a_i - y[i]
    dw1 += x[i][0] * dz_i  # 假设x[i][0]是特征x1
    dw2 += x[i][1] * dz_i  # 假设x[i][1]是特征x2
    db += dz_i

# 外部循环结束后，计算平均值
J /= m
dw1 /= m
dw2 /= m
db /= m

# 更新参数
w1 = w1 - alpha * dw1
w2 = w2 - alpha * dw2
b = b - alpha * db


上面这段代码中实现了一次梯度下降，也就是一次训练，但是使用了两个循环     
第一个循环是for循环遍历每一个样本     
第二个循环是对特征值进行循环。在这例子我们有 2 个特征值。如果你有超过两个特征时，需要循环 𝑑𝑤1 、 𝑑𝑤2 、 𝑑𝑤3 等等。



我们先来看**第二个循环的向量化**      
不用初始化 𝑑𝑤1 𝑑𝑤2 都等于 0，而是定义 𝑑𝑤 为一个向量，设置 $ w=np.zeros(n_x,1)  $  定义一个$n_x$行的一维向量    
其中 $n_x $代表单个样本的特征数，对于一张图片而言可能是64* 64 *3    
而其实dw不需要专门在代码开头初始化，因为dw是根据w自动算出来的，形状也是取决于w

<img src="images/第一层向量化.png">

再来看 **第一个循环的向量化**     
回忆一下最开始讲到的输入集


$X = [x^{(1)}, x^{(2)}, \dots, x^{(m)}]$: 表示不同的训练数据组成的输入矩阵，放在一个 $n_x \times m$ 的矩阵中;  
$n_x$ 是单个样本的特征点数，对于图片而言可能是64* 64 *3

$Y = [y^{(1)}, y^{(2)}, \dots, y^{(m)}]$: 对应训练不同训练数据组成的输出矩阵，维度为 $1 \times m$。

表示测试集的时候，我们会用 $M$ 来默认表示 $𝑀_{train}$  ,而测试集  $𝑀_{test}$ 需要单独注明：   

$M = [(x^{(1)}, y^{(1)}),(x^{(2)}, y^{(2)})], \dots, (x^{(m)}, y^{(m)})]  $



所以先计算 $z_1,z_2,z_3, \dots, z_n$  ,把他们都放到一个  $1 \times m$ 的行向量中    
你可以发现他可以表达为  $w^T$  (w的转置) $\times X+[b,b,\dots,b]$    
$[b,b,\dots,b]$ 是一个 $1 \times m$ 的行向量

所以计算的最终得到的$Z$是一个 $1 \times m$ 的向量，$Z = \begin{bmatrix} z^{(1)} & z^{(2)} & \ldots & z^{(m)} \end{bmatrix}$，计算方式为 $z = w^T X + \mathbf{b}$，其中 $\mathbf{b} = \begin{bmatrix} b & b & \ldots & b \end{bmatrix}$   

$$
Z = \begin{bmatrix}
w^T x^{(1)} + b \\
w^T x^{(2)} + b \\
\vdots \\
w^T x^{(m)} + b
\end{bmatrix}
$$

其中，
- $w^T x^{(1)} + b$ 是向量 $Z$ 的第一个元素，
- $w^T x^{(2)} + b$ 是第二个元素，
- 以此类推，直到 $w^T x^{(m)} + b$ 是第 $m$ 个元素。



In [ ]:
Z=np.dot(w.T, X)+b
# 其中b通过广播机制自动被拓展成一个 $1 \times m$ 的行向量

加下来要使用向量Z计算出向量Y    
$Y = [y^{(1)}, y^{(2)}, \dots, y^{(m)}]$: 对应训练不同训练数据组成的输出矩阵，维度为 $1 \times m$。

然后就可以计算 $ dZ =A-Y=[a^{(1)}-y^{(1)},a^{(2)}-y^{(2)},\dots, a^{(n)}-y^{(n)}   ]  $

对偏置 \(b\) 的梯度：
$$
\frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)})
$$
所以 $ db=\frac{1}{m}*\sum_{i=1}^m (dz^{(i)}) $   


In [ ]:
db=(1/m)*np.sum(dZ)

对权重 \(w\) 的梯度：
$$
\frac{\partial J}{\partial w} = \frac{1}{m} \sum_{i=1}^m (a^{(i)} - y^{(i)}) x^{(i)}
$$

所以  $ db=\frac{1}{m}*X*dz^T $   

### (五)总结

这是没有使用向量化的计算过程

<img src='images/原始计算过程.png'>

这是使用向量化之后的计算过程   


<img src='images/两层向量化.png'>


### (六)注意事项

这里要讲一个重要且常见的bug——一维数组

In [1]:
import numpy as np

a=np.random.rand(5)
print(a.shape)  #(5,)
# 此时a就是一个一维数组，既不是行向量也不是列向量

(5,)


这种结构很容易出现意想不到的bug，所以是要坚决摈弃的，一定要修改成(5,1)

In [4]:
import numpy as np

a=np.random.rand(5,1)
print(a.shape)  #(5,1)

(5, 1)


# 第二部分：浅层神经网络（Shallow neural networks）

### （一）浅层神经网络介绍

在上一部分，我们学习了二分类神经网络     
<img src='images/二分类神经网络.png'>

中间这个节点的计算过程如下：

$$
\begin{array}{c}
\textbf{x} \\
\textbf{w} \\
b \\
\end{array}
\rightarrow z = w^T x + b    \\
\rightarrow \alpha = \sigma(z)   \\
$$


这里我们将要学习浅层的神经网络   
<img src='images/浅层神经网络.png'>    




其中，我们用方框[1]表示第一层, [2]表示第二层，以此类推     
其中 [0] 也称为输入层，最后的 [2] 也称为输出层，中间的都叫做隐藏层。      
输入层是不算的，所以这一共有两层，所以叫“双层神经网络”    
在浅层的神经网络这幅图中，每一个神经节点也都是执行相同的操作      

以第一层的第一个节点为例
$$
\begin{array}{c}
\textbf{x} \\
W^{[1]} \\
b^{[1]} \\
\end{array}
\rightarrow z^{[1]} = W^{[1]} x + b^{[1]} \rightarrow a^{[1]} = \sigma(z^{[1]})
$$




但是正如图中所见，一层中是有多个神经元的    

<img src='images/双层神经网络.png'>


每一个节点最终都会输出一个预测值（或者叫激活值）a，就是我们在第一部分中的a    （ a 表示激活的意思）   
为了普遍化，我们直接把每个节点（包括输入层和输出层）的最终值都称为 a      
输入层的激活值称为 $a^{[0]}$，第一层的激活值记作 $a^{[1]}$。   
第一层的第一个单元或结点我们将其表示为 $a_1^{[1]}$，第二个结点的值我们记为 $a_2^{[1]}$ 以此类推。


$$
a^{[1]} = \begin{bmatrix}
a_1^{[1]} \\
a_2^{[1]} \\
a_3^{[1]} \\
a_4^{[1]}
\end{bmatrix}
$$


出于显而易见的向量化考虑，可以让$a^{[1]}$成为一个 $4 \times 1$ 的矩阵，$a^{[2]}$成为一个 $1 \times 1$ 的矩阵    
注意：   
**图中的 $x_1,x_2,x_3$ 等是指一个样本的不同特征,一共有$n_x$个，这里简化表达为3个，对于一张图片而言可能$n_x=64*64*3$**

同样的道理，每层都有每层的W参数和b参数，    

显而易见 $W^{[1]}$ 的形状是 $4 \times 3$,  $b^{[1]}$ 的形状是 $4 \times 1$    
（4是因为本层有4个节点，每个节点都有一个w，3是因为每个w都要处理3个样本特征，所以是3）   

同理可得 $W^{[2]}$ 的形状是 $1 \times 4$,  $b^{[2]}$ 的形状是 $1 \times 1$    
（1是因为本层有1个节点，每个节点都有一个w，4是因为每个w都要处理4个上一隐藏层的a的输入，所以是4）


### （二）浅层神经网络的计算

模仿二分类神经网络的计算，浅层神经网络的计算过程如下：       
每个小圆圈代表了计算的两个主要概念。

第一步，计算 $z_1^{[1]} = w_1^{[1]T} x + b_1^{[1]}$。

第二步，通过激活函数计算 $a_1^{[1]} = \sigma(z_1^{[1]})$。

随着层的第一个以及后面的每个神经元的计算过程一样，只是注意每层表示不同，层级分别对应 $a_2^{[1]}, a_3^{[1]}, a_4^{[1]}$，详细过程见下：

$$
\begin{align*}
z_1^{[1]} &= w_1^{[1]T} x + b_1^{[1]}, & a_1^{[1]} &= \sigma(z_1^{[1]}) \\
z_2^{[1]} &= w_2^{[1]T} x + b_2^{[1]}, & a_2^{[1]} &= \sigma(z_2^{[1]}) \\
z_3^{[1]} &= w_3^{[1]T} x + b_3^{[1]}, & a_3^{[1]} &= \sigma(z_3^{[1]}) \\
z_4^{[1]} &= w_4^{[1]T} x + b_4^{[1]}, & a_4^{[1]} &= \sigma(z_4^{[1]})
\end{align*}
$$


显而易见地可以使用向量化计算     
$$
Z^{[1]} = W^{[1]T} x + b^{[1]},  a^{[1]} = \sigma(Z^{[1]})   \\
\\
Z^{[n]} = W^{[n]T} x + b^{[n]},  a^{[n]} = \sigma(Z^{[n]})
$$

<img src='images/双层计算过程1.png'>

拓展到整个过程，就是如图    
<img src='images/双层计算过程2.png'>

同时，由于目前所有的计算都是针对单个样本的多个特征进行，所以还需要多个样本进行向量化      

**注：** 例如 $a^{[2](i)}$  (i) 是第i个训练样本，[2]是第二层

对于所有训练样本，需要让 𝑖 从 1到 𝑚实现下面这四个等式：

**计算第一层神经元的输出**：
- 计算线性组合 ：
$$
z^{[1](i)} = W^{[1]}(i) \cdot x^{(i)} + b^{[1]}(i)
$$
- 应用激活函数：
$$
a^{[1](i)} = \sigma(z^{[1](i)})
$$

**计算第二层神经元的输出**：
- 计算线性组合：
$$
z^{[2](i)} = W^{[2]}(i) \cdot a^{[1](i)} + b^{[2]}(i)
$$
- 应用激活函数：
$$
a^{[2](i)} = \sigma(z^{[2](i)})
$$



所以向量化之后总结如下：    

<img src='images/双层向量化.png'>



### （三）激活函数

sigmoid函数有时候并不是最好的激活函数。这里介绍4种激活函数

注：下面提到的“梯度消失”指x值很大的时候斜率很小


#### 1. Sigmoid 函数
- **公式**：$ \sigma(z) = \frac{1}{1 + e^{-z}} $
- **求导**：$ \sigma'(z) = \sigma(z)(1 - \sigma(z)) $
- **图像**：   
  <img src="images/sigmoid函数.png" width="300"/>
- **使用场景**：
  - 通常用于二分类问题中的输出层，因为它的输出范围是 (0,1)，可以表示概率。
  - 不太适用于隐藏层，因为在深层网络中容易引起梯度消失的问题。

#### 2. Tanh 函数
- **公式**：$ \tanh(z) = \frac{e^{z} - e^{-z}}{e^{z} + e^{-z}} $
- **求导**：$ \tanh'(z) = 1 - \tanh^2(z) $
- **图像**：  
  <img src="images/tanh函数.png" width="300"/>
- **使用场景**：
  - 常用于隐藏层，因为它的输出范围是 (-1,1)，比 sigmoid 函数的输出范围更广，可以更好地帮助模型学习。
  - 同样存在梯度消失的问题，尤其是在深层网络中。

#### 3. ReLU 函数
- **公式**：$ \text{ReLU}(z) = \max(0, z) $
- **求导**：
  - 当 $ z > 0 $ 时, $\text{ReLU}'(z) = 1$
  - 当 $ z \leq 0 $ 时, $\text{ReLU}'(z) = 0$
- **图像**：   
  <img src="images/ReLU函数.png" width="300"/>
- **使用场景**：
  - 非常流行用于各种网络的隐藏层，特别是在卷积神经网络中。
  - 有助于解决梯度消失问题，训练速度通常比 sigmoid 和 tanh 快。
  - 但存在死神经元问题，即一旦输入小于0，ReLU 激活的导数为0，该神经元可能再也不会对任何数据有激活现象了。

#### 4. Leaky ReLU 函数
- **公式**：$ \text{Leaky ReLU}(z) = \max(0.01z, z) $
- **求导**：
  - 当 $ z > 0 $ 时, $\text{Leaky ReLU}'(z) = 1$
  - 当 $ z \leq 0 $ 时, $\text{Leaky ReLU}'(z) = 0.01$
- **图像**：   
  <img src="images/Leaky ReLU函数.png" width="300"/>
- **使用场景**：
  - 用于解决 ReLU 死神经元问题，允许小的梯度当 \( z \leq 0 \) 时通过，提供所有神经元持续的梯度更新。
  - 适合用于深度学习网络中，特别是在面对复杂问题时，能够提高模型的稳定性。



### （四）浅层神经网络的梯度下降

我们用$n_x$表示输入特征的个数，所以根据不同的层数,$n^{[1]}$表示隐藏层单元个数，$n^{[2]}$表示输出单元个数

**神经网络权重和偏置的维度说明：**

- 第一层的权重 $ W^{[1]} $ 和偏置 $ b^{[1]} $ 的维度分别为 $ (n^{[1]}, n^{[0]}) $ 和 $ (n^{[1]}, 1) $，其中 $ n^{[0]} $ 是输入层的特征数量，$ n^{[1]} $ 是第一层的单元数或神经元数。
- 第二层的权重 $ W^{[2]} $ 和偏置 $ b^{[2]} $ 的维度分别为 $ (n^{[2]}, n^{[1]}) $ 和 $ (n^{[2]}, 1) $，其中 $ n^{[2]} $ 是第二层的单元数。

**loss function**：（与第一部分完全相同）

$$ 
L(\hat{y},y) = -ylog(\hat{y})-(1-y)log(1-\hat{y})
$$



**Cost function:**

$$
J(W^{[1]}, b^{[1]}, W^{[2]}, b^{[2]}) = \frac{1}{m} \sum_{i=1}^m L(\hat{y}, y)
$$




**Forward Propagation 正向传播:**


   $$
   z^{[1]} = W^{[1]} x + b^{[1]}
   $$

   $$
   a^{[1]} = \sigma(z^{[1]})
   $$

   $$
   z^{[2]} = W^{[2]} a^{[1]} + b^{[2]}
   $$

   $$
   a^{[2]} = g(z^{[2]}) = \sigma(z^{[2]})
   $$


**Back Propagation 反向传播:**


   $$
 \quad dz^{[2]} = a^{[2]} - Y, \quad Y = [y^{[1]} \dots y^{[m]}]
   $$

   $$
 \quad dW^{[2]} = \frac{1}{m} dz^{[2]} (a^{[1]})^T
   $$

   $$
 \quad db^{[2]} = \frac{1}{m} \text{np.sum}(dz^{[2]}, \text{axis} = 1, \text{keepdims} = \text{True})
   $$

   $$
 \quad dz^{[1]} = W^{[2]T} dz^{[2]} * g^{[1]'}(z^{[1]})
   $$

   $$
 \quad dW^{[1]} = \frac{1}{m} dz^{[1]} x^T
   $$


   $$
 \quad db^{[1]} = \frac{1}{m} \text{np.sum}(dz^{[1]}, \text{axis} = 1, \text{keepdims} = \text{True})
   $$

注意：
1. $g^{[1]'}$ 表示在隐藏层中使用的激活函数的导数。
2. axis=1表示水平相加求和
3. keepdims=True 是为了防止出现(n,)这种，确保输出为(n,1),或者使用reshape

**梯度下降：**    


$$
W^{[1]} = W^{[1]} - \alpha dW^{[1]}
$$

$$
b^{[1]} = b^{[1]} - \alpha db^{[1]}
$$

$$
W^{[2]} = W^{[2]} - \alpha dW^{[2]}
$$

$$
b^{[2]} = b^{[2]} - \alpha db^{[2]}
$$






**注意：这里只给出了公式，没有给出推导，没必要**


### （五）随机初始化

与之前的二分类神经网络不同，多层神经网络的权重W不能都是0    
因为如果 $W^{[1]}$ 和 $W^{[2]}$ 都是全为0的矩阵的话，就会导致这个神经网络每一个节点完全相同，完全对称，这就失去了神经网络的意义     
所以要对W进行高斯分布的随机初始化    
然而b是不需要的


<img src='images/随机初始化.png'>

In [ ]:
#对于上面这个神经网络而言，需要随机初始化如下
W_1=np.random.randn(2,2)*0.01
W_2=np.random.randn(2,2)*0.01
b_1=np.zeros((2,1))
b_2=0

为什么要乘以0.01呢？    
因为让权重小一点，对于隐藏层激活函数是tanh或炸sigmoid的，训练速度会快很多。   
因为中间斜率大

### (六)注意事项

1. **W1，W2等是不需要再向量化到一个矩阵中的**   
向量化的意思是：W1内部的多个同层节点的计算方法相同，所以可以保存到一个大矩阵W1中，同理推广     
然而对于W1和W2而言，他们作为权重，具有不同的形状，不同的激活函数，不同的正反向传播算法，所以是不可以放到一个向量中的     
对于n1，n2,还有b1,b2也是这个道理

2. **关于训练集和测试集**：    
注意训练集和测试集不能只包含猫，否则会导致它无法学习如何区分猫和非猫。这意味着你的模型实际上没有进行有效的学习来解决二分类问题，而只是学会了识别所有输入为同一类别。
在这种情况下，会导致输出的准确率变成100%

3. **关于隐藏层节点数的选择**：   
对于较简单的问题，较少的单元可能就足够了。   
对于复杂的问题，如复杂的图像识别或语音识别任务，可能需要更多的隐藏层和更多的单元     
过拟合：如果隐藏层单元太多，模型可能会过度学习训练数据的细节和噪声，导致在新的或未见过的数据上表现不佳。    
欠拟合：如果隐藏层单元太少，模型可能没有足够的能力来捕捉数据中的复杂模式，导致训练和测试性能都不佳。    
通常需要通过实验来找到合适的层数和每层的单元数。可以开始于一个相对较小的网络，逐渐增加单元数或层数，直到测试误差不再显著下降。   
还有一些经验法则如：   
a. 隐藏层单元数可以设置为输入层和输出层单元数的平均值。   
b. 可以尝试设置隐藏层单元数为输入特征数的2/3加上输出单元数。   


# 第三部分：深层神经网络（Deep Neural Networks） 

### （一）深层神经网络的介绍

<img src='images/四层神经网络.png'>

上图中我们看到一个典型的四层神经网络，对于每一层的节点数，可以这样表示：    
 
$$L=4 ， n^{[0]}=n^x=3 , n^{[1]}=5 , n^{[2]} =5 , n^{[3]}=3 ， n^{[4]}=n^{[L]}=1 $$

每一层l的激活函数记为 $a^{[l]}$, 其中第0层的$X=a^{[0]}$

下图解释了深度神经网络的内在机制，浅层隐藏层会去识别边缘，中层隐藏层识别部分特征，深层隐藏层可以构建一张完整人脸    

<img src='images/深层神经网络原理解释.png' width=500>

这种原理对于语言识别等神经网络也是适用的，比如浅层神经元处理小的音调升降，中间神经元识别音位（比如元音辅音），深层神经元识别单词，再形成词组，句子等

另外，相比于浅层神经网络，想要得到相同的计算结果，深层神经网络需要的总节点数量 比浅层神经网络 要少非常非常多（尤其是对于规模比较大的计算）

### (二)深度神经网络的正向和反向传播

   
对于l层而言：    

**正向传播：**

   
   $$
   Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}
   $$

   $$
   A^{[l]} = g^{[l]}(Z^{[l]})
   $$

其中对于第一层而言$A^{[l-1]}=A^{[0]}$就是$X$    

**反向传播：**
$$ 
dz^{[l]} = da^{[l]} \ast g'^{[l]}(z^{[l]})  \\
dw^{[l]} = dz^{[l]} \cdot a^{[l-1]}  \\
db^{[l]} = dz^{[l]}  \\
da^{[l-1]} = (w^{[l]})^T \cdot dz^{[l]}  \\
$$
向量化之后如下：
$$
dZ^{[l]} = dA^{[l]} \ast g'^{[l]}(Z^{[l]})   \\
dW^{[l]} = \frac{1}{m} dZ^{[l]} \cdot (A^{[l-1]})^T    \\
db^{[l]} = \frac{1}{m} \text{np.sum}(dZ^{[l]}, \text{axis} = 1, \text{keepdims} = True)  \\
dA^{[l-1]} = (W^{[l]})^T \cdot dz^{[l]}   \\
$$

整个计算流程如图

<img src='images/深层神经网络计算机制.png' width=600>

对于每一次训练，从X开始，向右正向传播，计算成本函数，然后反向传播    
从计算过程中可以看出，反向传播中需要正向传播的相应环节传递参数$Z^{[l]}$

但是仔细观察参数，我们可以发现有一个参数似乎没有得到计算，反向传播的第一个$dA^{[L]}$     
我们必须先算出这个值，   由于对于最后一层而言，     
$$ 
L(\hat{y},y) = -ylog(\hat{y})-(1-y)log(1-\hat{y})
$$
其中$y$是真实标签，$y^hat$是预测概率，也就是$A^{[L]}$   
所以要求的$dA^{[L]}$其实就是


$$
 dA^{[L]}=\frac{\partial L}{\partial A^{[L]}} = - \left( \frac{y}{A^{[L]}} - \frac{1 - y}{1 - A^{[L]}} \right) 
$$

用python表达式就是


In [ ]:
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

每一层的维度：   
$$
b^{[l]} : (n^{[l]},1)   \\
W^{[l]} : (n^{[l]},n^{[l-1]})  \\

$$

$dW^{[l]}$和$W^{[l]}$的维度相同，$db^{[l]}$和$b^{[l]}$的维度相同

对于z和a而言：    
**向量化前：**    
$$
Z^{[l]} : (n^{[l]},1)   \\
A^{[l]} : (n^{[l]},1)   \\
$$
**向量化后**:

$$
Z^{[l]} = (z^{[l](1)}, z^{ }, z^{ }, \ldots, z^{[l](m)})   \\
Z^{[l]} : (n^{[l]}, m)  \\
A^{[l]}: (n^{[l]}, m)    \\
A^{[0]} = X = (n^{[0]}, m)
$$


### （三）注意事项

1. 不是只有第一个W和b需要更新，每一个W和b都要更新！这样才是每一个神经元都得到训练！
2. 对于深度神经网络，在初始化W的时候，不要再使用*0.01来初始化权重W的标准差，这会导致神经网络在训练初期就陷入较小的梯度值（梯度消失问题）， cost会一直收敛在0.693147，也就是-log(0.5)。对于ReLU激活函数，建议使用He初始化。    He 初始化有助于神经网络更快地收敛。这是因为它避免了权重在训练初期过小或过大，这些极端值可能会导致学习过程缓慢或不稳定。
3. 对于深度神经网络，仅仅是100张照片是远远不够的，虽然理论上可以用很少的图片（如几百张）来训练一个模型，但这通常会导致模型性能不佳或者过拟合。有些来源建议至少需要1000张图片每个类别来训练一个稳健的模型 。数据的多样性和质量同样重要。不仅需要数量足够，还需要保证数据能够覆盖到类别的各种变化，包括不同的环境、角度、光照条件等。
